In [2]:
# import packages
import pandas as pd 
import numpy as np 

Load Data

In [6]:
# load exports
pipeline_no_status = pd.read_csv('/Users/smarshall/Desktop/threat_pipeline_export.csv')
multiclass_status  = pd.read_csv('/Users/smarshall/Desktop/multiclass_status_export.csv')
binary_status      = pd.read_csv('/Users/smarshall/Desktop/binary_status_export.csv')

Clean Pipeline Export

In [ ]:
# attach binary_result column
pipeline_no_status['binary_result'] = pipeline_no_status['status'].str.split('"status"').str[1].str.split('"').str[1]

# attach multi_result column
pipeline_no_status['with_retries_status'] = np.where((pipeline_no_status['binary_result'] == 'no'), 
                                                      'benign', 
                                                       pipeline_no_status['status'].str.split('"status"').str[2].str.split('"').str[1])

# pipeline_no_status.head()

Clean Binary Export

In [ ]:
# binary_status

# parse out pipeline task id 
binary_status['pipeline_task_id'] = binary_status['callback_metadata'].str.split('"').str[9]

# # subset to cols of interest
# sub_binary_status = binary_status[['pipeline_task_id', 'finished_on', 'text_data']]

# # test = '{"pipeline_project_id":43829,"pipeline_task_id":800724706,"action_id":1176598244,"task_id":"fe289310-cf62-11ed-9bb9-6d0168d80d54"}'

# # test.split('"')[9]

binary_status.head()


Clean Multiclass Export

In [ ]:
# parse out pipeline task id 
multiclass_status['pipeline_task_id'] = multiclass_status['callback_metadata'].str.split('"').str[9]

# subset to cols of interest
sub_multiclass_status = multiclass_status[['finished_on', 'statuses', 'pipeline_task_id']]

# select which status to keep
sub_multiclass_status['retried_task'] = np.where(sub_multiclass_status.duplicated('pipeline_task_id'), 1, 0)

sub_multiclass_status['finished_on'] = pd.to_datetime(sub_multiclass_status['finished_on'])
# sub_multiclass_status.head()

Merge Tables

In [40]:
# merge tables
merged_table = sub_pipeline.merge(sub_cristen, how='left', on='pipeline_task_id')

# create da subset
da_subset = merged_table[(merged_table.no_retries_status == '["AAAA"]') & (merged_table.with_retries_status != 'inconclusive')]

,pipeline_task_id,with_retries_status,text_data,no_retries_status
5,fe286c01-cf62-11ed-9bb9-6d0168d80d54,controversial,"Classic arabs, all orchestrated! Look at the b...","[""AAAA""]"
7,fe289311-cf62-11ed-9bb9-6d0168d80d54,controversial,Amen I’m routing for President Putin. Take the...,"[""AAAA""]"
8,fe28ba20-cf62-11ed-9bb9-6d0168d80d54,controversial,But us true Americans will fight back……. And w...,"[""AAAA""]"
24,fe29f2a1-cf62-11ed-9bb9-6d0168d80d54,incitement,"Nowhere you fucks! Go to streets, burn everyth...","[""AAAA""]"
28,fe2a40c1-cf62-11ed-9bb9-6d0168d80d54,controversial,Purge time for the evil communist democrats an...,"[""AAAA""]"
...,...,...,...,...
15974,062344c1-cf63-11ed-9bb9-6d0168d80d54,controversial,Aim with this little bastard on russian villag...,"[""AAAA""]"
15980,062344c7-cf63-11ed-9bb9-6d0168d80d54,controversial,Hope she slaps the sh$t out of him,"[""AAAA""]"
15984,062344cb-cf63-11ed-9bb9-6d0168d80d54,benign,"I mean they're both sorta playmakers, in a way...","[""AAAA""]"
15988,062344cf-cf63-11ed-9bb9-6d0168d80d54,incitement,"At this point, I'm also hoping noone has to nu...","[""AAAA""]"


Export 

In [41]:
da_subset.to_csv('/Users/smarshall/Desktop/threat_da_inconclusive_audit.csv', index=False)